In [1]:
import matplotlib.pyplot as plt 
import pandas as pd
import math
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GRU 

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import psycopg2
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt 
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'white'
import seaborn as sns

import keras_functions
from keras_functions import *

Using TensorFlow backend.


In [2]:
con = psycopg2.connect(database='codeforces', user='Joy')
cur = con.cursor()

In [3]:
binvars = get_categorical_variables([
    'all_participanttypes',
    'all_tags',
    'all_language'
])

# Initial pipeline

In [4]:
with open('set_train.txt') as f:
    train_handles = [h.strip() for h in f.readlines()]
with open('set_test.txt') as f:
    test_handles = [h.strip() for h in f.readlines()]

In [41]:
user_batch = 2#len(test_handles)

In [96]:
from os.path import exists
reload(keras_functions)
X = []
Y = []
maxt = 0
lens = [0] * user_batch
runsum = 0

for i in range(user_batch):
    try:
        filename = 'rnn_train/' + train_handles[i] + ".csv"
        #if i%100 == 0:
        #print i, filename
        if not exists(filename):
            continue
        x, y, t, colnames = keras_functions.get_user_data(train_handles[i], binvars=binvars, month=3, maxtimepts=5000)

        if x is None:
            continue
        maxt = max(t, maxt)
        X.append(x)
        Y.append(y)
        runsum += x.shape[0]
        lens[i] = runsum
    except:
        print "ERROR AT", i, filename
        break

# DEBUGGING 

In [97]:
list(colnames).index('hours_submit_to_solve')

111

In [98]:
colnames

array(['CONTESTANT', 'GNU C++', 'MEMORY_LIMIT_EXCEEDED', 'OK', 'PRACTICE',
       'RUNTIME_ERROR', 'TIME_LIMIT_EXCEEDED', 'VIRTUAL', 'WRONG_ANSWER',
       'contestid', 'delta_smoothed_3months', 'newrating', 'oldrating',
       'points', 'problem_rating', 'rank', 'smoothed_3months',
       'shortest paths', '2-sat', 'greedy', 'meet-in-the-middle',
       'matrices', 'number theory', 'constructive algorithms',
       'chinese remainder theorem', 'implementation', 'ternary search',
       'schedules', 'dsu', 'dfs and similar', 'graph matchings',
       'string suffix structures', 'math', 'probabilities', 'fft',
       'divide and conquer', 'two pointers', 'trees', 'data structures',
       'flows', 'sortings', 'expression parsing', 'dp', 'hashing',
       'bitmasks', '*special', 'geometry', 'combinatorics', 'graphs',
       'brute force', 'games', 'binary search', 'strings',
       'PRESENTATION_ERROR', 'TESTING', 'CRASHED', 'CHALLENGED',
       'REJECTED', 'FAILED', 'SKIPPED', 'IDLENESS

In [108]:
list(X[1][0][:20, 107])

[240.27944444444444,
 243.03,
 265.55444444444447,
 219.19305555555556,
 264.39888888888891,
 270.01055555555558,
 157.60499999999999,
 241.48444444444445,
 156.24166666666667,
 195.29472222222222,
 266.0361111111111,
 241.34666666666666,
 242.24388888888888,
 218.71111111111111,
 219.02277777777778,
 269.77027777777778,
 -1.0,
 122.39583333333333,
 222.07166666666666,
 239.85972222222222]

In [65]:
reload(keras_functions)
debug = keras_functions.get_user_data(train_handles[i], binvars=binvars, month=3, maxtimepts=5000)
#debug[0][['ratingupdatetimeseconds', 'solvetimeseconds', 'starttimeseconds', 'stoptimeseconds', 'hours_solve_to_contest', 'hours_submit_to_solve']]
debug[0][['hours_solve_to_contest', 'hours_submit_to_solve']]

,hours_solve_to_contest,hours_submit_to_solve
0,240.279444,0.062500
23,243.030000,0.000000
24,265.554444,0.108889
27,219.193056,0.000000
28,264.398889,0.000000
30,270.010556,0.194722
32,157.605000,0.000000
34,241.484444,0.000000
35,156.241667,0.000000
36,195.294722,0.000000


In [71]:
maxt

46

In [ ]:
X_batch = np.concatenate(X, axis=0)

In [ ]:
X_batch.shape

In [ ]:
Y

In [ ]:
epochs = 5

In [ ]:
for e in range(epochs):
    